# MCP Experiment with OOP Concepts

This notebook demonstrates basic Model Context Protocol (MCP) concepts using Object-Oriented Programming (OOP) principles in Python.

## 1. Core MCP Classes

Define the fundamental classes representing MCP components: `MCPTool`, `MCPResource`, `MCPServer`, and `MCPExperiment`.

In [ ]:
from typing import Any, Dict, List

class MCPTool:
    """Represents a tool provided by an MCP server."""
    def __init__(self, tool_name: str, description: str, parameters: Dict[str, str]):
        self.tool_name = tool_name
        self.description = description
        self.parameters = parameters # Expected parameters and their types/descriptions
        print(f"Tool '{self.tool_name}' initialized.")

    def execute(self, input_args: Dict[str, Any]) -> Dict[str, Any]:
        """Simulates executing the tool. Needs to be implemented by subclasses."""
        print(f"Executing tool '{self.tool_name}' with args: {input_args}")
        # Basic validation (can be expanded)
        for param in self.parameters:
            if param not in input_args:
                return {"error": f"Missing required parameter: {param}"}
        
        # Placeholder for actual execution logic
        result = self._run(input_args)
        print(f"Tool '{self.tool_name}' execution finished.")
        return result
    
    def _run(self, input_args: Dict[str, Any]) -> Dict[str, Any]:
        """Protected method for subclass implementation of tool logic."""
        raise NotImplementedError("Subclasses must implement the _run method.")

class MCPResource:
    """Represents a resource provided by an MCP server."""
    def __init__(self, resource_uri: str, resource_type: str):
        self.resource_uri = resource_uri
        self.resource_type = resource_type
        print(f"Resource '{self.resource_uri}' ({self.resource_type}) initialized.")

    def fetch_data(self) -> Any:
        """Simulates fetching data from the resource. Needs to be implemented by subclasses."""
        print(f"Fetching data for resource '{self.resource_uri}'...")
        data = self._load_data()
        print(f"Data fetched for resource '{self.resource_uri}'.")
        return data
    
    def _load_data(self) -> Any:
        """Protected method for subclass implementation of data loading logic."""
        raise NotImplementedError("Subclasses must implement the _load_data method.")

class MCPServer:
    """Represents an MCP server hosting tools and resources."""
    def __init__(self, server_name: str):
        self.server_name = server_name
        self.tools: Dict[str, MCPTool] = {}
        self.resources: Dict[str, MCPResource] = {}
        print(f"Server '{self.server_name}' initialized.")

    def register_tool(self, tool: MCPTool):
        """Adds a tool to the server."""
        if tool.tool_name in self.tools:
            print(f"Warning: Tool '{tool.tool_name}' already registered. Overwriting.")
        self.tools[tool.tool_name] = tool
        print(f"Tool '{tool.tool_name}' registered to server '{self.server_name}'.")

    def add_resource(self, resource: MCPResource):
        """Adds a resource to the server."""
        if resource.resource_uri in self.resources:
            print(f"Warning: Resource '{resource.resource_uri}' already exists. Overwriting.")
        self.resources[resource.resource_uri] = resource
        print(f"Resource '{resource.resource_uri}' added to server '{self.server_name}'.")
        
    def use_tool(self, tool_name: str, arguments: Dict[str, Any]) -> Dict[str, Any]:
        """Executes a registered tool."""
        if tool_name not in self.tools:
            return {"error": f"Tool '{tool_name}' not found on server '{self.server_name}'."}
        tool = self.tools[tool_name]
        return tool.execute(arguments)

    def access_resource(self, resource_uri: str) -> Any:
        """Fetches data from a registered resource."""
        if resource_uri not in self.resources:
            return {"error": f"Resource '{resource_uri}' not found on server '{self.server_name}'."}
        resource = self.resources[resource_uri]
        return resource.fetch_data()

class MCPExperiment:
    """Manages and runs an MCP experiment involving multiple servers."""
    def __init__(self, experiment_title: str):
        self.experiment_title = experiment_title
        self.servers: Dict[str, MCPServer] = {}
        print(f"\n--- Starting Experiment: {self.experiment_title} ---")

    def add_server(self, server: MCPServer):
        """Adds an MCP server to the experiment."""
        if server.server_name in self.servers:
            print(f"Warning: Server '{server.server_name}' already added. Overwriting.")
        self.servers[server.server_name] = server
        print(f"Server '{server.server_name}' added to experiment '{self.experiment_title}'.")
        
    def get_server(self, server_name: str) -> MCPServer | None:
        """Retrieves a server by name."""
        return self.servers.get(server_name)

    def run_tests(self):
        """Placeholder for running predefined tests."""
        print(f"\n--- Running Tests for Experiment: {self.experiment_title} ---")
        # Test logic will be added later
        print("No tests defined yet.")
        print(f"--- Test Run Complete ---")

    def generate_report(self):
        """Placeholder for generating an experiment report."""
        print(f"\n--- Generating Report for Experiment: {self.experiment_title} ---")
        print(f"Experiment Title: {self.experiment_title}")
        print(f"Servers Involved: {list(self.servers.keys())}")
        # More detailed report generation can be added
        print(f"--- Report Generation Complete ---")

# Example Usage (Initialization)
print("Initializing core MCP classes...")
experiment = MCPExperiment("Basic OOP Demo")
print("Core classes defined.")

## 2. Math Server Example

Now, let's create a concrete implementation: a `MathServer` with a `calculate` tool and a `constants` resource.

In [ ]:
import math

# --- Concrete Tool Implementation ---
class CalculateTool(MCPTool):
    def __init__(self):
        super().__init__(
            tool_name="calculate",
            description="Performs basic arithmetic operations.",
            parameters={
                "operation": "['add', 'subtract', 'multiply', 'divide']",
                "a": "number",
                "b": "number"
            }
        )

    def _run(self, input_args: Dict[str, Any]) -> Dict[str, Any]:
        op = input_args['operation']
        a = input_args['a']
        b = input_args['b']
        
        try:
            a_num = float(a)
            b_num = float(b)
        except ValueError:
            return {"error": "Parameters 'a' and 'b' must be numbers."}

        if op == 'add':
            result = a_num + b_num
        elif op == 'subtract':
            result = a_num - b_num
        elif op == 'multiply':
            result = a_num * b_num
        elif op == 'divide':
            if b_num == 0:
                return {"error": "Division by zero."}
            result = a_num / b_num
        else:
            return {"error": f"Unknown operation: {op}"}
            
        return {"result": result}

# --- Concrete Resource Implementation ---
class ConstantsResource(MCPResource):
    def __init__(self):
        super().__init__(resource_uri="math/constants", resource_type="dict")
        self._data = {
            "pi": math.pi,
            "e": math.e
        }

    def _load_data(self) -> Dict[str, float]:
        # In a real scenario, this might load from a file or external source
        return self._data

# --- Concrete Server Implementation ---
class MathServer(MCPServer):
    def __init__(self):
        super().__init__(server_name="math_server")
        # Register the specific tools and resources for this server
        self.register_tool(CalculateTool())
        self.add_resource(ConstantsResource())

# --- Add MathServer to the Experiment ---
print("\nInitializing Math Server Example...")
math_server = MathServer()
experiment.add_server(math_server)
print("Math Server example initialized and added to the experiment.")

## 3. Interactive Demo

Use the defined classes and the `MathServer` instance to interact with the simulated MCP components.

In [ ]:
# Get the math server from the experiment
server = experiment.get_server("math_server")

if server:
    # Example 1: Use the 'calculate' tool (Addition)
    print("\n--- Example 1: Using 'calculate' tool (Add) ---")
    add_result = server.use_tool(
        tool_name="calculate", 
        arguments={"operation": "add", "a": 10, "b": 5}
    )
    print(f"Addition Result: {add_result}")

    # Example 2: Use the 'calculate' tool (Division)
    print("\n--- Example 2: Using 'calculate' tool (Divide) ---")
    div_result = server.use_tool(
        tool_name="calculate", 
        arguments={"operation": "divide", "a": 10, "b": 2}
    )
    print(f"Division Result: {div_result}")

    # Example 3: Access the 'constants' resource
    print("\n--- Example 3: Accessing 'constants' resource ---")
    constants = server.access_resource(resource_uri="math/constants")
    print(f"Constants: {constants}")

    # Example 4: Tool execution with missing parameter
    print("\n--- Example 4: Tool error (Missing parameter) ---")
    error_result = server.use_tool(
        tool_name="calculate", 
        arguments={"operation": "multiply", "a": 7} # Missing 'b'
    )
    print(f"Error Result: {error_result}")

    # Example 5: Accessing a non-existent resource
    print("\n--- Example 5: Resource error (Not found) ---")
    non_existent_resource = server.access_resource(resource_uri="math/formulas")
    print(f"Non-existent Resource Result: {non_existent_resource}")
else:
    print("Error: Math server not found in the experiment.")

## 4. Validation Tests (Placeholder)

This section is intended for more structured tests, including edge cases and error handling.

In [ ]:
# Run the placeholder test function from the experiment object
experiment.run_tests()

# TODO: Add specific test cases here, e.g.:
# - Test division by zero
# - Test invalid operation name
# - Test non-numeric inputs for 'a' and 'b'
# - Test accessing non-existent tools/resources

## 5. Report Generation (Placeholder)

Generate a summary report of the experiment.

In [ ]:
# Generate the placeholder report
experiment.generate_report()